# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'human_nontata_promoters'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "demo_human_or_worm"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_human_or_worm 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

2021-10-22 23:54:24.138810: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-22 23:54:24.138826: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-10-22 23:54:26.890716: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-22 23:54:26.890762: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-10-22 23:54:26.891362: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

Dataset `demo_human_or_worm` has 2 classes: human, worm.

All lenghts of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:96: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


,train,test
human,37500,12500
worm,37500,12500


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 75000 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-10-22 23:54:28.513317: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 'a', 't', 'c', 'g', 'n']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


   1/1172 [..............................] - ETA: 14:09 - loss: 0.8723 - binary_accuracy: 0.4062

   6/1172 [..............................] - ETA: 13s - loss: 0.7721 - binary_accuracy: 0.4922  

  11/1172 [..............................] - ETA: 13s - loss: 0.7457 - binary_accuracy: 0.5014

  16/1172 [..............................] - ETA: 13s - loss: 0.7134 - binary_accuracy: 0.5420

...

1160/1172 [============================>.] - ETA: 0s - loss: 0.1316 - binary_accuracy: 0.9475

1164/1172 [============================>.] - ETA: 0s - loss: 0.1317 - binary_accuracy: 0.9476

1168/1172 [============================>.] - ETA: 0s - loss: 0.1315 - binary_accuracy: 0.9476

1172/1172 [==============================] - ETA: 0s - loss: 0.1315 - binary_accuracy: 0.9476

1172/1172 [==============================] - 17s 14ms/step - loss: 0.1315 - binary_accuracy: 0.9476


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 25000 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

  1/391 [..............................] - ETA: 1:21 - loss: 0.0626 - binary_accuracy: 1.0000

 11/391 [..............................] - ETA: 1s - loss: 0.1540 - binary_accuracy: 0.9418  

 22/391 [>.............................] - ETA: 1s - loss: 0.1577 - binary_accuracy: 0.9375

 33/391 [=>............................] - ETA: 1s - loss: 0.1495 - binary_accuracy: 0.9422

 45/391 [==>...........................] - ETA: 1s - loss: 0.1546 - binary_accuracy: 0.9385

...

361/391 [==========================>...] - ETA: 0s - loss: 0.1543 - binary_accuracy: 0.9407

374/391 [===========================>..] - ETA: 0s - loss: 0.1549 - binary_accuracy: 0.9404

387/391 [============================>.] - ETA: 0s - loss: 0.1546 - binary_accuracy: 0.9407

391/391 [==============================] - 2s 4ms/step - loss: 0.1554 - binary_accuracy: 0.9403


[0.1554224193096161, 0.9403200149536133]